In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pyhive import presto
from pymongo import MongoClient
import warnings
warnings.filterwarnings("ignore")
import my_func

presto_conn = presto.connect(
    host='presto.processing.yoda.run',
    port=80,
    protocol='http',
    catalog='hive',
    username='mayank.jha@rapido.bike',
    # requests_kwargs=req_kw,
)

In [15]:
start_date = '2021-11-01'
end_date = '2021-11-14'
city = ['Delhi','Hyderabad','Mumbai','Bangalore','Chennai','Kolkata','Jaipur']

In [446]:
df_segment = pd.DataFrame()
for day in date_list :
    city = ['Delhi','Hyderabad','Mumbai','Bangalore','Chennai','Kolkata','Jaipur']
    print('Running for timeperiod : ', day, ' to ', (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d'))
    for ct in city:
        print('    City : ', ct)
        sd = day.replace('-','')
        ed = (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d').replace('-','')
        %run Segments.py '{sd}' '{ed}' '{ct}'
        segment = pd.read_csv('Captain_Segment.csv')
        segment['city'] = ct
        segment['week'] = pd.to_datetime(day).strftime('%Y-%V')[-2:]
        segment['week'] = segment['week'].apply(pd.to_numeric)
        df_segment = pd.concat([df_segment, segment])

Running for timeperiod :  2021-10-04  to  2021-10-10
    City :  Delhi
    City :  Hyderabad
    City :  Mumbai
    City :  Bangalore
    City :  Chennai
    City :  Kolkata
    City :  Jaipur


In [193]:
df_segment['week'].unique()

array([34, 35, 36, 37, 38, 39, 40])

In [232]:
df_segment[df_segment['city']=='Hyderabad'][['RIDER','Segment','Performance','week']].to_csv('x.csv', index=False)

In [21]:
df_segment.head()

,RIDER,MEDIAN_DBR,ADBR,NUM_OBS,captain_id,active_days,Performance,Opportunity,Actual_Captain_cancellations,Rider_cancellation,Captain_induced_CC,RPR,TrueDPR,CCR_Rate,Consistency,Segment,city,week
0,573f29249b0ffc283677a933,3.0,1.500000,2,573f29249b0ffc283677a933,2,1.0,4.0,1.0,1.0,0.0,1.5,0.200000,0.500000,intra_week,LP_Intra_LO,Delhi,34
1,5b40a0a3499d526d0aa1b252,0.0,0.102564,5,5b40a0a3499d526d0aa1b252,5,5.0,5.0,0.0,0.0,0.0,5.0,1.000000,0.000000,daily,MP_D_MO,Delhi,34
2,5b9a82b06636546559dc406f,0.0,0.333333,4,5b9a82b06636546559dc406f,4,3.0,6.0,0.0,0.0,0.0,4.0,0.454545,0.142857,daily,LP_D_MO,Delhi,34
3,5bf2a235fe82667f3f01336e,0.0,0.000000,1,5bf2a235fe82667f3f01336e,1,2.0,4.0,0.0,0.0,0.0,2.0,0.500000,0.000000,daily,LP_D_LO,Delhi,34
4,5c21c968d9bacb2f6f9e87cb,0.0,0.666667,2,5c21c968d9bacb2f6f9e87cb,2,2.0,3.0,1.0,1.0,0.0,2.0,0.833333,0.166667,daily,LP_D_LO,Delhi,34


In [449]:
df_segment.to_csv('top_7_city_segment_6_weeks.csv', index=False)

In [230]:
df_segment = pd.read_csv('top_7_city_segment_6_weeks.csv')

In [101]:
def get_quality_segments(start, city):
    
    print("fetching quality Segments data for",start)

    orders_query = """select captain_id, 
        city_name, 
        performance, 
        opportunity, 
        consistency, 
        rpr, 
        dpr, 
        true_dpr, 
        dapr, 
        true_dapr, 
        apr, 
        true_apr, 
        ccr_rate, 
        active_days, 
        daily_incentive_earnings, 
        weekly_incentive_earnings, 
        order_earnings, 
        login_hours_daily, 
        rph, 
        segment, 
        recency_segment, 
        ltr,
        CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week
    from datasets.captain_quality_segments
    where day_of_week(DATE(date_parse(yyyymmdd,'%Y%m%d'))) = 7
        and yyyymmdd = '{sd}'
        and city_name in {ct}
               """.format( sd = start, ct = city)
    
    #print(orders_query)

    df_orders_data = pd.read_sql(orders_query, presto_conn)
    
    print(" fetching completed for quality segments data", len(df_orders_data))

    return df_orders_data

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 6]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

quality_segment = pd.DataFrame()

for day in date_list:
    week_rf = get_quality_segments(day.replace('-',''), "('Delhi','Hyderabad','Mumbai','Bangalore','Chennai','Kolkata','Jaipur')")
    quality_segment = pd.concat([quality_segment, week_rf])
    
quality_segment.head()

fetching quality Segments data for 20210912
 fetching completed for quality segments data 91287
fetching quality Segments data for 20210919
 fetching completed for quality segments data 96252
fetching quality Segments data for 20210926
 fetching completed for quality segments data 100073
fetching quality Segments data for 20211003
 fetching completed for quality segments data 102934
fetching quality Segments data for 20211010
 fetching completed for quality segments data 104728
fetching quality Segments data for 20211017
 fetching completed for quality segments data 102595


,captain_id,city_name,performance,opportunity,consistency,rpr,dpr,true_dpr,dapr,true_dapr,...,active_days,daily_incentive_earnings,weekly_incentive_earnings,order_earnings,login_hours_daily,rph,segment,recency_segment,ltr,week
0,5f8edf7415b87f96738748a2,Mumbai,0,2,daily,2.0,0.000000,0.000000,0.000000,0.000000,...,1,15.0,0.0,38.784,2.931944,0.00,LP_D_LO,dormant_LP_D_LO,643.0,36
1,5fa51722edd1e59c45bc18a7,Mumbai,4,8,daily,4.0,0.333333,0.416667,0.500000,0.666667,...,2,25.0,0.0,35.896,2.677222,1.49,MP_D_HO,dormant_MP_D_HO,84.0,36
2,5fa9601a9a911c6ee8722231,Mumbai,1,2,intra_week,1.0,0.500000,0.500000,0.666667,1.000000,...,8,10.0,0.0,44.056,1.050556,0.95,LP_Intra_LO,active_LP_Intra_LO,74.0,36
3,5fd37b2d8e3f058566b4b294,Mumbai,1,2,inter_week,1.0,0.500000,0.500000,1.000000,1.000000,...,1,0.0,0.0,61.584,0.589167,1.70,LP_Inter_LO,inactive_LP_Inter_LO,72.0,36
4,604288546a8a76232387ca2e,Mumbai,0,3,daily,10.0,0.000000,0.000000,0.000000,0.000000,...,5,15.0,80.0,36.008,0.822222,0.00,LP_D_LO,active_LP_D_LO,45.0,36


In [82]:
quality_segment['segment'].unique()

array(['LP_Intra_HO', 'LP_Intra_UHO', 'HP_D_UHO', 'MP_D_HO',
       'LP_Intra_LO', 'LP_Inter_MO', 'MP_D_MO', 'LP_D_HO', 'MP_D_UHO',
       'LP_D_LO', 'LP_Intra_MO', 'LP_Inter_LO', 'HP_D_MO', 'LP_Inter_HO',
       'LP_D_UHO', 'LP_Inter_UHO', 'HP_D_HO', 'LP_D_MO', 'UHP_D_UHO',
       'MP_D_LO', 'UHP_D_HO'], dtype=object)

In [84]:
quality_segment['full_part_timer'] = quality_segment['segment'].apply(lambda x : 'full_timer' if any(y in x for y in ['HP_D_HO','HP_D_UHO','UHP_D_HO','UHP_D_UHO']) 
                                                            else 'consistent_part_timer' if any(y in x for y in ['MP_D_HO','LP_D_LO','LP_D_MO','MP_D_MO','LP_D_HO','MP_D_UHO','LP_D_UHO','MP_D_LO']) 
                                                            else 'casual_part_timer')
#df_segment['combined_segment'] = df_segment['Segment']
quality_segment.head()

,captain_id,city_name,performance,opportunity,consistency,rpr,dpr,true_dpr,dapr,true_dapr,...,daily_incentive_earnings,weekly_incentive_earnings,order_earnings,login_hours_daily,rph,segment,recency_segment,ltr,week,full_part_timer
0,5745a655be4237ee7658b675,Bangalore,0,7,intra_week,2.0,0.000000,0.000000,0.000000,0.000000,...,10.0,0.0,25.728,2.126111,0.00,LP_Intra_HO,dormant_LP_Intra_HO,108.0,38,casual_part_timer
1,5af60eedb24946086dc6d340,Bangalore,1,24,intra_week,1.0,0.034483,0.043478,0.142857,1.000000,...,8.0,0.0,40.000,3.116944,0.32,LP_Intra_UHO,dormant_LP_Intra_UHO,13.0,38,casual_part_timer
2,5b29f7749dcf013f42865763,Bangalore,8,12,daily,10.0,0.566667,0.700000,0.636364,0.846154,...,15.0,250.0,56.330,5.538611,1.44,HP_D_UHO,active_HP_D_UHO,1700.0,38,full_timer
3,5b4631ef6349957febcb7875,Bangalore,3,8,daily,3.0,0.272727,0.333333,0.714286,1.000000,...,15.0,0.0,40.000,0.140833,21.30,MP_D_HO,dormant_MP_D_HO,95.0,38,consistent_part_timer
4,5b499b76a2389052823f79a5,Bangalore,1,3,intra_week,1.0,0.250000,0.333333,0.500000,1.000000,...,15.0,0.0,34.152,0.532500,1.88,LP_Intra_LO,active_LP_Intra_LO,31.0,38,casual_part_timer


In [22]:

def get_rf_segments(start):
    
    print("fetching rf Segments data for",start)

    orders_query = """SELECT captainid as rider, day, frequency_segment, recency_segment, segment as cu_segment
                      from datasets.captain_cu_immutable
                      where day = date('{sd}')
                      and city = 'Hyderabad'
                      --AND segment = 'RF'                      
               """.format( sd = start)
    
#     print(orders_query)

    df_orders_data = pd.read_sql(orders_query, presto_conn)
    
    print(" fetching completed for rf segments data", len(df_orders_data))

    return df_orders_data

date_list = [pd.to_datetime(start_date)-timedelta(days=1) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)-timedelta(days=1)).days)
        if (pd.to_datetime(start_date)-timedelta(days=1) + timedelta(days=x)).weekday() == 6]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]
date_list

rf_segments = pd.DataFrame()

for day in date_list:
    week_rf = get_rf_segments(day)
    rf_segments = pd.concat([rf_segments, week_rf])
    
    


rf_segments['recency_segment'] = rf_segments['recency_segment'].apply(lambda x : x if pd.isna(x) else x.title())
rf_segments['frequency_segment'] = rf_segments['frequency_segment'].apply(lambda x : x if pd.isna(x) else x.title())
rf_segments['rf_segment'] = rf_segments['recency_segment'] +'-'+ rf_segments['frequency_segment']
rf_segments['week'] = pd.to_datetime(rf_segments['day']).apply(lambda x : ((x+timedelta(days=1)).strftime('%Y-%V')[-2:]))
rf_segments['week'] = rf_segments['week'].apply(pd.to_numeric)
rf_segments.head()

fetching rf Segments data for 2021-10-31
 fetching completed for rf segments data 259817
fetching rf Segments data for 2021-11-07
 fetching completed for rf segments data 261295


,rider,day,frequency_segment,recency_segment,cu_segment,rf_segment,week
0,5f183c94dc80dd407572e5dd,2021-10-31,None,Inactive,CHURNED,NaN,44
1,5d62ef94d0286d106d6404fe,2021-10-31,None,Inactive,CHURNED,NaN,44
2,5ca21a808c352421ea06e835,2021-10-31,None,Inactive,CHURNED,NaN,44
3,5d4c15d2bde10e0d5433119d,2021-10-31,None,Inactive,CHURNED,NaN,44
4,5b392bc45862d70533e8fb5e,2021-10-31,None,Inactive,CHURNED,NaN,44


In [128]:
rf_segments.to_csv('x.csv', index=False)

In [11]:
rf_segments = pd.read_csv('quality_segment_rf_segment.csv')

In [164]:
df_captains = df_segment.merge(rf_segments, how = 'left', left_on = ['RIDER','week'], right_on = ['rider','week'])

In [226]:
level_query = """select captainid, mobilenumber from datasets.captain_single_view where registeredcity = 'chennai' or lastridecity = 'Chennai' """

df_mobile = pd.read_sql(level_query, presto_conn)
print(len(df_captains))

12799


In [228]:
df_mobile.to_csv('x.csv', index=False)

In [458]:
df_mobile.to_csv('mobile_number.csv', index=False)

In [459]:
df_captains = df_captains.merge(df_mobile, how = 'left', left_on = 'RIDER', right_on = 'captainid')

In [102]:
df_captains = quality_segment.copy()

In [99]:
df_captains.pivot_table(index='week', columns='segment', values='captain_id', aggfunc = 'nunique')#.to_csv('x.csv')

week,38,39,40,41
segment,,,,
HP_D_HO,36.785254,36.576002,36.676742,36.928227
HP_D_UHO,36.339391,36.030541,36.357203,37.126936
LP_D_HO,38.139717,37.963593,38.111799,38.274518
LP_D_LO,37.179701,37.108457,37.149782,37.478675
LP_D_MO,37.527960,37.485354,37.531332,38.115183
LP_D_UHO,39.836082,39.628412,39.656656,40.521351
LP_Inter_HO,46.862127,47.933594,49.791751,48.768714
LP_Inter_LO,44.300916,43.284487,43.917494,44.869028
LP_Inter_MO,42.815455,46.398464,45.929912,46.649232


In [90]:
# Retention Calculation WoW with week 34 as reference

df_temp = df_captains.copy()
retention = pd.DataFrame(columns=['city_name','segment','week','retention'])
city = ['Hyderabad']
for c in city:
    for s in df_temp[df_temp['city_name']==c]['segment'].unique():
        for w in [39, 40, 41]:
            #print('city :',c, ' segment : ', s, ' week : ', w)
            reference = df_temp[(df_temp['city_name']==c) & (df_temp['week']==38) & (df_temp['segment']==s)]
            forward = df_temp[(df_temp['city_name']==c) & (df_temp['week']==w)]
            combined = reference.merge(forward, how = 'inner', on = 'captain_id')
            r = combined['captain_id'].nunique()/reference['captain_id'].nunique()*100 if reference['captain_id'].nunique() != 0 else 0
            x = pd.DataFrame({"city_name" : [c], "segment" : [s], "week" : [w], "retention" : [r]})
            retention = pd.concat([retention, x])
retention.head()

,city_name,segment,week,retention
0,Hyderabad,HP_D_HO,39,89.346110
0,Hyderabad,HP_D_HO,40,79.706877
0,Hyderabad,HP_D_HO,41,67.249154
0,Hyderabad,MP_D_MO,39,90.183464
0,Hyderabad,MP_D_MO,40,81.177031


In [93]:
retention

,city_name,segment,week,retention
0,Hyderabad,HP_D_HO,39,89.346110
0,Hyderabad,HP_D_HO,40,89.957653
0,Hyderabad,HP_D_HO,41,86.384072
0,Hyderabad,MP_D_MO,39,90.183464
0,Hyderabad,MP_D_MO,40,91.198157
0,Hyderabad,MP_D_MO,41,87.242798
0,Hyderabad,LP_D_HO,39,90.000000
0,Hyderabad,LP_D_HO,40,89.461713
0,Hyderabad,LP_D_HO,41,90.033223
0,Hyderabad,MP_D_HO,39,90.637967


In [77]:
retention.to_csv('x.csv', index=False)

In [78]:
df_captains.pivot_table(index=['city','Segment'], columns=['week'], values='RPR', aggfunc = 'mean').to_csv('x.csv')

In [92]:
# Retention Calculation WoW

df_temp = df_captains.copy()
retention = pd.DataFrame(columns=['city_name','segment','week','retention'])
city = ['Hyderabad']
for c in city:
    for s in df_temp[df_temp['city_name']==c]['segment'].unique():
        for w in [38,39,40]:
            #print('city :',c, ' segment : ', s, ' week : ', w)
            reference = df_temp[(df_temp['city_name']==c) & (df_temp['week']==w) & (df_temp['segment']==s)]
            forward = df_temp[(df_temp['city_name']==c) & (df_temp['week']==w+1)]
            combined = reference.merge(forward, how = 'inner', on = 'captain_id')
            r = combined['captain_id'].nunique()/reference['captain_id'].nunique()*100 if reference['captain_id'].nunique() != 0 else 0
            x = pd.DataFrame({"city_name" : [c], "segment" : [s], "week" : [w+1], "retention" : [r]})
            retention = pd.concat([retention, x])
retention.head()

,city_name,segment,week,retention
0,Hyderabad,HP_D_HO,39,89.346110
0,Hyderabad,HP_D_HO,40,89.957653
0,Hyderabad,HP_D_HO,41,86.384072
0,Hyderabad,MP_D_MO,39,90.183464
0,Hyderabad,MP_D_MO,40,91.198157


In [82]:
retention.to_csv('x.csv', index=False)

In [ ]:
# Segment Movement

df_temp = df_captains[df_captains['city']=='Hyderabad'].copy()
week = [34,35,36,37,38,39]
loop = 1
for s in df_temp[df_temp['city']==c]['Segment'].unique():
    for w in week[:-1]:
        for we in week[loop:len(week)]:
        reference = df_temp[(df_temp['week']==w) & (df_temp['Segment']==s)]
        forward = df_temp[df_temp['week']==we]
        combined = reference.merge(forward, how = 'inner', on = 'RIDER')
        combined['Segment']

In [294]:
df_temp = df_captains[df_captains['city']=='Chennai'][['RIDER','Segment','week']]
week = [34,35,36,37,38,39]
loop = 1
combined = pd.DataFrame()
for s in ['MP_D_MO']:
    for w in [34]:
        for we in [35,36,37,38,39]:
            reference = df_temp[(df_temp['week']==w) & (df_temp['Segment']==s)]
            forward = df_temp[df_temp['week']==we]
            x = reference.merge(forward, how = 'inner', on = 'RIDER')
            combined = pd.concat([combined, x])

In [295]:
combined.pivot_table(index='Segment_y', columns = 'week_y', values = 'RIDER', aggfunc = 'nunique')

week_y,35,36,37,38,39
Segment_y,,,,,
HP_D_HO,79.0,78.0,51.0,40.0,42.0
HP_D_UHO,6.0,9.0,5.0,4.0,1.0
LP_D_HO,4.0,5.0,7.0,3.0,6.0
LP_D_LO,160.0,130.0,167.0,167.0,161.0
LP_D_MO,73.0,66.0,55.0,41.0,60.0
LP_Inter_HO,NaN,NaN,NaN,NaN,1.0
LP_Inter_LO,30.0,24.0,34.0,22.0,22.0
LP_Inter_MO,2.0,2.0,2.0,1.0,3.0
LP_Intra_LO,18.0,27.0,20.0,28.0,27.0


In [252]:
x = df_temp[(df_temp['week']==34) & (df_temp['Segment']=='HP_D_UHO')]
y = df_temp[(df_temp['week']==35)]
z = x.merge(y, how = 'inner', on = 'RIDER')
z.head()

,RIDER,MEDIAN_DBR_x,ADBR_x,NUM_OBS_x,captain_id_x,active_days_x,Performance_x,Opportunity_x,Actual_Captain_cancellations_x,Rider_cancellation_x,...,orders_earnings_y,riderid_y_y,daily_incentive_y,weekly_incentive_y,rider_y,link_orders_y,delivery_orders_y,other_orders_y,riderid_y,_col2_y
0,607ac2d4c1137373c3b8daea,0.0,0.090909,5,607ac2d4c1137373c3b8daea,5,14.0,17.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5d1c18c83b752c45cf941dfb,0.0,0.000000,1,5d1c18c83b752c45cf941dfb,1,9.0,17.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5dbf2ce23087fd4936338e77,0.0,0.043796,7,5dbf2ce23087fd4936338e77,7,16.0,29.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5dd24f7a0856f14618371570,0.0,0.063830,7,5dd24f7a0856f14618371570,7,13.0,18.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ce4d948377155163c018c1a,0.0,0.084746,5,5ce4d948377155163c018c1a,5,15.0,17.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df_captains.head()

,RIDER,MEDIAN_DBR,ADBR,NUM_OBS,captain_id,active_days,Performance,Opportunity,Actual_Captain_cancellations,Rider_cancellation,...,Segment,city,week,full_part_timer,rider,day,frequency_segment,recency_segment,cu_segment,rf_segment
0,573f29249b0ffc283677a933,3.0,1.500000,2,573f29249b0ffc283677a933,2,1.0,4.0,1.0,1.0,...,LP_Intra_LO,Delhi,34,casual_part_timer,573f29249b0ffc283677a933,2021-08-29,Underperforming,Recent,RF,Recent-Underperforming
1,5b40a0a3499d526d0aa1b252,0.0,0.102564,5,5b40a0a3499d526d0aa1b252,5,5.0,5.0,0.0,0.0,...,MP_D_MO,Delhi,34,consistent_part_timer,5b40a0a3499d526d0aa1b252,2021-08-29,Midperforming,Recent,RF,Recent-Midperforming
2,5b9a82b06636546559dc406f,0.0,0.333333,4,5b9a82b06636546559dc406f,4,3.0,6.0,0.0,0.0,...,LP_D_MO,Delhi,34,consistent_part_timer,5b9a82b06636546559dc406f,2021-08-29,Midperforming,Recent,RF,Recent-Midperforming
3,5bf2a235fe82667f3f01336e,0.0,0.000000,1,5bf2a235fe82667f3f01336e,1,2.0,4.0,0.0,0.0,...,LP_D_LO,Delhi,34,consistent_part_timer,5bf2a235fe82667f3f01336e,2021-08-29,Midperforming,Recent,RF,Recent-Midperforming
4,5c21c968d9bacb2f6f9e87cb,0.0,0.666667,2,5c21c968d9bacb2f6f9e87cb,2,2.0,3.0,1.0,1.0,...,LP_D_LO,Delhi,34,consistent_part_timer,5c21c968d9bacb2f6f9e87cb,2021-08-29,Midperforming,Recent,RF,Recent-Midperforming


In [121]:
df_captains[df_captains['city']=='Chennai'].pivot_table(index=['Segment','cu_segment'], columns=['week'], values='RIDER', aggfunc = 'nunique')##.to_csv('x.csv')

week                         34      35      36      37      38      39
Segment      cu_segment                                                
HP_D_HO      HH           162.0   187.0   213.0   228.0   183.0   169.0
             RF          1119.0  1243.0  1124.0  1053.0   864.0   990.0
HP_D_UHO     HH           152.0   121.0   136.0   118.0   117.0    90.0
             RF           599.0   605.0   548.0   451.0   393.0   472.0
LP_D_HO      CHURNED        NaN     NaN     NaN     NaN     1.0     NaN
             HH           129.0   160.0   153.0   144.0   158.0   153.0
             RF           436.0   567.0   581.0   573.0   549.0   622.0
LP_D_LO      CHURNED        NaN     NaN     NaN     NaN     1.0     2.0
             HH           254.0   322.0   308.0   393.0   414.0   312.0
             RF          1582.0  1760.0  1811.0  2212.0  2197.0  2115.0
LP_D_MO      HH           290.0   323.0   292.0   309.0   351.0   278.0
             RF          1121.0  1281.0  1352.0  1539.0  1499.0  1466.0
LP_D_UHO     CHURNED        NaN     NaN     NaN     NaN     NaN     1.0
             HH            62.0    52.0    61.0    53.0    53.0    43.0
             RF           257.0   295.0   291.0   248.0   199.0   299.0
LP_Inter_HO  HH            28.0    31.0    23.0    48.0    34.0    25.0
             RF            54.0    57.0    49.0    42.0    56.0    51.0
LP_Inter_LO  CHURNED        NaN     NaN     NaN     NaN     1.0     NaN
             HH           194.0   250.0   227.0   269.0   295.0   254.0
             RF           708.0   820.0   884.0   941.0   918.0   899.0
LP_Inter_MO  CHURNED        NaN     NaN     NaN     1.0     NaN     NaN
             HH            46.0    81.0    81.0    69.0    73.0    67.0
             RF           144.0   136.0   160.0   166.0   152.0   143.0
LP_Inter_UHO HH             9.0    12.0    12.0    11.0     9.0    10.0
             RF            20.0    17.0    16.0    18.0    10.0    17.0
LP_Intra_HO  HH            10.0    13.0    12.0     9.0    17.0    11.0
             RF            43.0    51.0    68.0    71.0    60.0    53.0
LP_Intra_LO  CHURNED        NaN     NaN     NaN     NaN     NaN     1.0
             HH            52.0    66.0    49.0    88.0    78.0    68.0
             RF           478.0   487.0   474.0   656.0   651.0   581.0
LP_Intra_MO  HH            23.0    35.0    29.0    40.0    38.0    25.0
             RF           146.0   162.0   174.0   204.0   199.0   157.0
LP_Intra_UHO HH             2.0     4.0     6.0     5.0     2.0     2.0
             RF            36.0    34.0    28.0    26.0    24.0    26.0
MP_D_HO      HH           462.0   486.0   535.0   474.0   521.0   448.0
             RF          1812.0  1797.0  1761.0  1906.0  1733.0  2020.0
MP_D_LO      HH            27.0    29.0    36.0    38.0    34.0    31.0
             RF            99.0   123.0   133.0   158.0   143.0   162.0
MP_D_MO      HH           316.0   405.0   396.0   446.0   448.0   385.0
             RF          1451.0  1631.0  1705.0  2028.0  1905.0  2060.0
MP_D_UHO     HH           140.0   139.0   128.0   129.0   114.0   111.0
             RF           277.0   271.0   282.0   287.0   271.0   353.0
MP_Intra_HO  RF             NaN     NaN     NaN     NaN     1.0     NaN
UHP_D_UHO    HH            12.0     7.0    16.0    10.0    11.0    11.0
             RF            96.0   102.0    85.0    88.0    87.0    99.0

In [13]:
df_temp = df_captains[df_captains['city']=='Hyderabad']
reference = df_temp[(df_temp['week']==34) & (df_temp['Segment']=='HP_D_HO')]
forward = df_temp[(df_temp['week']==39) & (df_temp['Segment']=='HP_D_HO')]
comb = reference.merge(forward, how = 'outer', on ='RIDER', indicator = True)
ret = comb[comb['_merge']=='both']
not_ret = comb[comb['_merge']=='left_only']

In [17]:
ret.head()

,RIDER,MEDIAN_DBR_x,ADBR_x,NUM_OBS_x,captain_id_x,active_days_x,Performance_x,Opportunity_x,Actual_Captain_cancellations_x,Rider_cancellation_x,...,city_y,week_y,full_part_timer_y,rider_y,day_y,frequency_segment_y,recency_segment_y,cu_segment_y,rf_segment_y,_merge
0,5a0ec089cfece61ab6e0eda4,0.0,0.090909,5.0,5a0ec089cfece61ab6e0eda4,5.0,9.0,12.0,1.0,0.0,...,Hyderabad,39.0,full_timer,5a0ec089cfece61ab6e0eda4,2021-10-03,Midperforming,Recent,RF,Recent-Midperforming,both
10,5d617550d0286d106d622062,0.0,0.135135,4.0,5d617550d0286d106d622062,4.0,10.0,14.0,2.0,1.0,...,Hyderabad,39.0,full_timer,5d617550d0286d106d622062,2021-10-03,Highperforming,Recent,RF,Recent-Highperforming,both
32,5ffc2edd857d1b56a628fd44,0.0,0.176471,2.0,5ffc2edd857d1b56a628fd44,2.0,9.0,11.0,1.0,0.0,...,Hyderabad,39.0,full_timer,5ffc2edd857d1b56a628fd44,2021-10-03,Midperforming,Recent,RF,Recent-Midperforming,both
35,60732b1fde2a0a461e25b336,0.0,0.117647,5.0,60732b1fde2a0a461e25b336,5.0,10.0,14.0,2.0,1.0,...,Hyderabad,39.0,full_timer,60732b1fde2a0a461e25b336,2021-10-03,Midperforming,Recent,RF,Recent-Midperforming,both
65,5ce92685377155163c053c31,0.0,0.108108,4.0,5ce92685377155163c053c31,4.0,10.0,12.0,1.0,0.0,...,Hyderabad,39.0,full_timer,5ce92685377155163c053c31,2021-10-03,Midperforming,Recent,RF,Recent-Midperforming,both


In [25]:
ret.pivot_table(index='full_part_timer_x', values = 'active_days_x', aggfunc = 'mean')

,active_days_x
full_part_timer_x,
full_timer,4.328


In [26]:
not_ret.pivot_table(index='full_part_timer_x', values = 'active_days_x', aggfunc = 'mean')

,active_days_x
full_part_timer_x,
full_timer,3.528547


In [24]:
# Login hours

def get_login_hours(start_date, end_date, city):
    print("fetching Login hours for : ",start_date," to ", end_date)
    orders_query = """   SELECT 
            userid as rider, 
            CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week,
            --date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') AS logindate, 
            cast(sum(duration) as double)/cast((60*60*1000) as double) as login_hours,
            SUM(CASE WHEN quarter_hour >= '0600' and quarter_hour <= '1159' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS MORNING_DURATION,
            SUM(CASE WHEN quarter_hour >= '1200' and quarter_hour <= '1759' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS AFTERNOON_DURATION,
            SUM(CASE WHEN quarter_hour >= '1800' and quarter_hour <= '2359' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS EVENING_DURATION
        FROM hive.datasets.captain_login_hours
        WHERE yyyymmdd >= '{sd}' AND yyyymmdd <= '{ed}'
        AND status in ('2','3','6','7','8','10')
        AND userid in (SELECT captainId from datasets.captain_single_view WHERE (registeredcity in {ct} or lastridecity in {ct})  AND activationdate is not null)
        GROUP BY 1,2             
               """.format(sd = start_date.replace('-',''), ed = end_date.replace('-',''), ct = city)
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched Login hours", len(df_lh))

    return df_lh

df_login_hours = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    login_hours = get_login_hours(day, (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d'), "('chennai')")
    x = pd.to_datetime(day).strftime('%Y-%V')
    #login_hours['week'] = x[-2:]
    #login_hours['week'] = login_hours['week'].apply(pd.to_numeric)
    df_login_hours = pd.concat([df_login_hours,login_hours])

df_login_hours.head()

fetching Login hours for :  2021-11-01  to  2021-11-07
 fetched Login hours 16792
fetching Login hours for :  2021-11-08  to  2021-11-14
 fetched Login hours 16742


,rider,week,login_hours,MORNING_DURATION,AFTERNOON_DURATION,EVENING_DURATION
0,6071a54266dcdebd1f61b0c6,44,12.212222,0.000000,6.600278,5.611944
1,61384da18709d25b8296f6b3,44,0.005556,0.005556,0.000000,0.000000
2,607ad544255e27650bbd347f,44,3.421389,0.216667,1.966667,1.238056
3,5d45c7a3f3dbe16ba31df4f0,44,2.199722,1.034444,1.165278,0.000000
4,5cc31e3454bc7263ff508766,44,0.078056,0.078056,0.000000,0.000000


In [276]:
df_login_hours.to_csv('x.csv', index=False)

In [25]:
# Earnings

def get_earnings():

    orders_query = """   
    select riderid, 
        week,
        sum(amount) as orders_earnings
    from
        (select distinct riderId as riderid,
                orderid,
                CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week,
                cast(totalearning as double) as amount 
            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
            where yyyymmdd between '{sd}' and '{ed}'
                and city in {ct} 
                and transactionType = 'orders' 
                and status = 'success'
        )
    group by 1,2
               """.format(sd = start_date.replace('-',''), ed = end_date.replace('-',''), ct = "('Chennai')")
    
    #print(orders_query)
    df_orders_data = pd.read_sql(orders_query, presto_conn)
    return df_orders_data

df_earnings = get_earnings()
df_earnings.head()

,riderid,week,orders_earnings
0,6167dd353cf262293f196119,45,201.3300
1,5d13648cddfc0234287fdbdc,45,1905.5390
2,5fa8e8f62e35451bbd47335f,45,1423.8605
3,61820113fbff14a50ff873c6,45,650.1820
4,6159a57c82a4579bcf9040c6,45,1473.2880


In [12]:
df_earnings.to_csv('x.csv', index=False)

In [57]:
# Find the incentive eligible and incentive achieved

def get_incentive_eligible(start_date, end_date, city):
    
    print("fetching eligible incentive for : ",start_date," to ", end_date)

    incentive_eligible = """with incentive as 
            (select distinct _id as incentive_id
                from hive.raw.mongodb_rapidopayroll_incentives_immutable
                where json_array_contains(cities, '{ct}')
                    and startDate between '{sd}' and '{ed}'
                    and endDate between '{sd}' and '{ed}'
            ),
daily_incentive as
            (select riderid,
                    --yyyymmdd,
                    sum(daily_incentive_achieved) as daily_incentive
                from
                    (select distinct riderid,
                            yyyymmdd,
                            tincentiveIdl,
                            subIncentiveId,
                            incentivestage,
                            cast(amount as double) as daily_incentive_achieved
                        from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                        where incentivetype = 'Daily' 
                            and status = 'success' 
                            and yyyymmdd between '{sd1}' and '{ed1}'
                            and city = '{ct}'
                    ) x
                    join incentive as i
                        on x.tincentiveIdl = i.incentive_id
                group by 1
            ),
weekly_incentive as
            (select riderid,
                    --yyyymmdd,
                    sum(weekly_incentive_achieved) as weekly_incentive
                from
                    (select distinct riderid,
                            yyyymmdd,
                            tincentiveIdl,
                            subIncentiveId,
                            incentivestage,
                            cast(amount as double) as weekly_incentive_achieved
                        from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                        where incentivetype = 'Weekly Fixed' 
                            and status = 'success' 
                            and yyyymmdd between '{sd1}' and '{ed1}'
                            and city = '{ct}'
                    ) x
                    join incentive as i
                        on x.tincentiveIdl = i.incentive_id
                group by 1
            )
            select coalesce(d.riderid, w.riderid) as riderid,
                --coalesce(d.yyyymmdd, w.yyyymmdd) as yyyymmdd,
                d.daily_incentive,
                w.weekly_incentive
            from daily_incentive as d
                full outer join 
                    weekly_incentive as w
                        on d.riderid = w.riderid --and d.yyyymmdd = w.yyyymmdd                  
               """.format(sd = start_date, ed = end_date, sd1 = start_date.replace('-',''), ed1 = end_date.replace('-',''),ct = city)
    
    #print(incentive_eligible)

    df_incentive_eligible = pd.read_sql(incentive_eligible, presto_conn)
    
    print(" fetching incentive eligible captains", len(df_incentive_eligible))

    return df_incentive_eligible

df_incentive_eligible = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    incentive_eligible = get_incentive_eligible(day, (pd.to_datetime(day) + timedelta(7)).strftime('%Y-%m-%d'), 'Chennai')
    x = pd.to_datetime(day).strftime('%Y-%V')
    incentive_eligible['week'] = x[-2:]
    incentive_eligible['week'] = incentive_eligible['week'].apply(pd.to_numeric)
    df_incentive_eligible = pd.concat([df_incentive_eligible,incentive_eligible])

df_incentive_eligible.head()

fetching eligible incentive for :  2021-11-01  to  2021-11-08
 fetching incentive eligible captains 3287
fetching eligible incentive for :  2021-11-08  to  2021-11-15
 fetching incentive eligible captains 3782


,riderid,daily_incentive,weekly_incentive,week
0,60251886b778656069abd6f7,147.0,220.0,44
1,616e29fbfbce665d059c3e85,40.0,NaN,44
2,608503f0c14f0fd018ad0b89,154.0,100.0,44
3,614b1b44de864c60190408c8,25.0,NaN,44
4,5d038b1c8819fb622ea13df4,50.0,NaN,44


In [15]:
df_incentive_eligible.to_csv('x.csv', index=False)

In [202]:
# Find the penalty

def get_penalty(start_date, end_date, city):
    
    print("fetching penalty for : ",start_date," to ", end_date)

    incentive_eligible = """select riderid, 
        week, 
        sum(amount) as penalty
    from
        (select riderid, 
                CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week, 
                cast(amount as double) amount, 
                dense_rank() over(partition by _id order by updated_epoch desc) as rnk 
            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable 
            where yyyymmdd between '{sd}' and '{ed}'
                and transactiontype = 'penalty' 
                and status = 'success' 
                and city = '{ct}'
        ) x
    where x.rnk = 1
    group by 1,2                   
               """.format(sd = start_date, ed = end_date, sd1 = start_date.replace('-',''), ed1 = end_date.replace('-',''),ct = city)
    
    #print(incentive_eligible)

    x = pd.read_sql(incentive_eligible, presto_conn)
    
    print(" fetching incentive eligible captains", len(x))

    return x

df_penalty = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    penalty = get_penalty(day.replace('-',''), (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d').replace('-',''), 'Hyderabad')
    x = pd.to_datetime(day).strftime('%Y-%V')
    penalty['week'] = x[-2:]
    penalty['week'] = penalty['week'].apply(pd.to_numeric)
    df_penalty = pd.concat([df_penalty, penalty])

df_penalty.head()

fetching eligible incentive for :  20210823  to  20210829
 fetching incentive eligible captains 1263
fetching eligible incentive for :  20210830  to  20210905
 fetching incentive eligible captains 1432
fetching eligible incentive for :  20210906  to  20210912
 fetching incentive eligible captains 3847
fetching eligible incentive for :  20210913  to  20210919
 fetching incentive eligible captains 1748
fetching eligible incentive for :  20210920  to  20210926
 fetching incentive eligible captains 1639
fetching eligible incentive for :  20210927  to  20211003
 fetching incentive eligible captains 1713


,riderid,week,_col2
0,5bfd5587d76a02356e91ff15,34,-11.55065
1,5ced12de377155163c0888a9,34,-18.91285
2,5d20639c668011467e2a0653,34,-29.31950
3,5f40726dd358d170654fd29e,34,-30.00000
4,5cb5cd0254bc7263ff3cb2dd,34,-19.72020


In [27]:
# Rides & Ratings

rides_query = """SELECT rider,
                    --orderdate,
                    CAST(WEEK(DATE(date_parse(orderdate,'%Y-%m-%d'))) as int) as week,
                    sum(case when serviceobj_service = 'Link' then 1 else 0 end) as link_orders,
                    sum(case when serviceobj_service = 'Delivery' then 1 else 0 end) as delivery_orders,
                    sum(case when serviceobj_service not in ('Link','Delivery') then 1 else 0 end) as other_orders,
                    count(distinct orderdate) as active_days
                    from legacy.orders
                    where orderdate >= '{sd}'
                    and orderdate <= '{ed}'
                    and status = 'dropped'
                    and spdfraud_flag != 1
                    and serviceobj_city in {ct}
                    group by 1, 2""".format(sd=start_date, ed= end_date, ct = "('Chennai')")

df_rides = pd.read_sql(rides_query, presto_conn)
df_rides.head()

,rider,week,link_orders,delivery_orders,other_orders,active_days
0,5fde19d04319a780bab1ce34,45,0,0,14,5
1,613b76338709d27754992cb3,44,0,0,11,2
2,5f8d1764b711633f3ae0b00b,44,0,0,2,2
3,5d53cdc555fbf50d45f444bb,45,0,29,11,6
4,5fb515c4b275d9dc598f3567,45,4,0,0,2


In [6]:
df_rides.to_csv('x.csv', index=False)

In [58]:
df_test = pd.read_csv('mas_w45_test.csv')
df_test['tag'] = 'Test'
df_control = pd.read_csv('mas_w45_control.csv')
df_control['tag'] = 'Control'

In [59]:
df_captains = pd.concat([df_test, df_control])

In [60]:
df_captains['full_part_timer'] = df_captains['segment'].apply(lambda x : 'full_timer' if any(y in x for y in ['HP_D_HO','HP_D_UHO','HP_D_MO','UHP_D_HO','UHP_D_UHO']) 
                                                            else 'consistent_part_timer' if any(y in x for y in ['MP_D_UHO','MP_D_HO','MP_D_MO','MP_D_LO','LP_D_LO','LP_D_MO','LP_D_HO','LP_D_UHO',]) 
                                                            else 'casual_part_timer')

In [62]:
df_captains = df_captains.merge(df_login_hours[df_login_hours['week']==45], how = 'left', left_on = ['rider'], right_on = ['rider'])

In [63]:
df_captains = df_captains.merge(df_earnings[df_earnings['week']==45], how = 'left', left_on = ['rider'], right_on = ['riderid'])

In [64]:
df_captains = df_captains.merge(df_incentive_eligible[df_incentive_eligible['week']==45], how = 'left', left_on = ['rider'], right_on = ['riderid'])

In [61]:
df_captains = df_captains.merge(df_rides[df_rides['week']==45], how = 'left', left_on = ['rider'], right_on = ['rider'])

In [66]:
df_captains.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6768 entries, 0 to 6767
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mobilenumber        6768 non-null   int64  
 1   rider               6768 non-null   object 
 2   segment             6768 non-null   object 
 3   tag                 6768 non-null   object 
 4   full_part_timer     6768 non-null   object 
 5   week_x              2421 non-null   float64
 6   link_orders         2421 non-null   float64
 7   delivery_orders     2421 non-null   float64
 8   other_orders        2421 non-null   float64
 9   active_days         2421 non-null   float64
 10  week_y              3180 non-null   float64
 11  login_hours         3180 non-null   float64
 12  MORNING_DURATION    3180 non-null   float64
 13  AFTERNOON_DURATION  3180 non-null   float64
 14  EVENING_DURATION    3180 non-null   float64
 15  riderid_x           2421 non-null   object 
 16  week_x

In [141]:
df_captains[df_captains['link_orders']>0].pivot_table(index='segment', columns = 'tag', values = 'rider', aggfunc = 'nunique')

tag,Control,Test
segment,,
Dormant,2.0,NaN
Evening_Only_LP_D_HO,11.0,12.0
Evening_Only_LP_D_LO,117.0,128.0
Evening_Only_LP_D_MO,72.0,94.0
Evening_Only_MP_D_HO,88.0,120.0
Evening_Only_MP_D_MO,118.0,127.0
HP_D_HO,123.0,138.0
HP_D_UHO,69.0,59.0
Intra_Inter,111.0,110.0


In [69]:
df_captains['total_incentive'] = df_captains['daily_incentive'] + df_captains['weekly_incentive']

In [128]:
df_captains['link_orders'] = df_captains['link_orders'].replace(np.nan, 0)

In [84]:
%run Segments.py '20211108' '20211114' 'Chennai'
segment = pd.read_csv('Captain_Segment.csv')

In [138]:
segment_movement = pd.DataFrame(columns=['segment','rider','segment_maintain','segment_upgrade','segment_downgrade','churn'])
for s in ['LP_D_HO']:
    x = df_captains[(df_captains['segment']==s) & (df_captains['tag']=='Test')]
    retained = len(x[(x['new_segment']==s) & (x['link_orders']>0)])/len(x)*100.0
    upgrade = len(x[(x['new_segment'].isin(['UHP_D_UHO','HP_D_UHO','HP_D_HO','MP_D_UHO','MP_D_HO','MP_D_MO','MP_D_LO','LP_D_UHO'])) & (x['link_orders']>0)])/len(x)*100.0
    #downgrade = len(x[(x['new_segment'])  & (x['link_orders']>0)])/len(x)*100.0
    churned = len(x[x['link_orders']==0])/len(x)*100.0
    print('Total : ',len(x))
    print('Retained : ', retained)
    print('Upgrade : ', upgrade)
    #print('Downgrade : ', downgrade)
    print('Churned : ', churned)

Total :  63
Retained :  0.0
Upgrade :  4.761904761904762
Churned :  90.47619047619048


In [125]:
len(x)

28

In [139]:
df_captains[(df_captains['segment']=='LP_D_HO') & (df_captains['tag']=='Test')]#.to_csv('x.csv', index=False)

,mobilenumber,rider,segment,tag,full_part_timer,week_x,link_orders,delivery_orders,other_orders,active_days,...,EVENING_DURATION,riderid_x,week_x,orders_earnings,riderid_y,daily_incentive,weekly_incentive,week_y,total_incentive,new_segment
2927,9087859024,5c66f909bbf6957649422e07,LP_D_HO,Test,consistent_part_timer,45.0,0.0,1.0,0.0,1.0,...,1.149167,5c66f909bbf6957649422e07,45.0,40.0000,NaN,NaN,NaN,NaN,NaN,No Segment
2928,8220728785,5ddbc7a9a80df9312a19ba84,LP_D_HO,Test,consistent_part_timer,45.0,0.0,13.0,3.0,2.0,...,8.019167,5ddbc7a9a80df9312a19ba84,45.0,670.0000,5ddbc7a9a80df9312a19ba84,96.0,NaN,45.0,NaN,No Segment
2929,9677277983,5d30383e84e7394e947180cd,LP_D_HO,Test,consistent_part_timer,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Segment
2930,8248954733,605cb12470424a21faf5f933,LP_D_HO,Test,consistent_part_timer,45.0,2.0,0.0,0.0,2.0,...,0.958056,605cb12470424a21faf5f933,45.0,58.2165,NaN,NaN,NaN,NaN,NaN,LP_Intra_LO
2931,8610126183,617d0b9bc32482b9a4e2cad3,LP_D_HO,Test,consistent_part_timer,45.0,10.0,0.0,0.0,2.0,...,3.402778,617d0b9bc32482b9a4e2cad3,45.0,395.7175,617d0b9bc32482b9a4e2cad3,65.0,100.0,45.0,165.0,MP_D_HO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985,7904884090,60b33462270b9c90e2dd92ad,LP_D_HO,Test,consistent_part_timer,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Segment
2986,9003045942,614965b9e8f223f381ab826c,LP_D_HO,Test,consistent_part_timer,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Segment
2987,6381643100,5d6fa048d0286d106d771db1,LP_D_HO,Test,consistent_part_timer,NaN,0.0,NaN,NaN,NaN,...,0.007500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Segment
2988,9080255581,60f96dc78f347987781c0a47,LP_D_HO,Test,consistent_part_timer,NaN,0.0,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No Segment


In [242]:
df_temp = df_captains[df_captains['city']=='Chennai']

In [230]:
# For Hyderabad and HP_D_UHO segment

w34 = df_temp[(df_temp['Segment']=='HP_D_HO') & (df_temp['week']==34)]
w35 = df_temp[(df_temp['week']==35)]
w34 = w34.merge(w35, how = 'outer', on = 'RIDER', indicator = True)
constant = w34[(w34['_merge']=='both') & (w34['Segment_y']=='HP_D_UHO')]
churned = w34[w34['_merge']=='left_only']
upgrade = w34[(w34['_merge']=='both') & (w34['Segment_y'].isin(['UHP_D_UHO']))]
downgrade = w34[(w34['_merge']=='both') & (~w34['Segment_y'].isin(['HP_D_UHO','UHP_D_UHO']))]

In [237]:
constant['RIDER'].nunique()

373

In [236]:
upgrade['RIDER'].nunique()

130

In [238]:
downgrade['RIDER'].nunique()

459

In [239]:
churned['RIDER'].nunique()

319

In [241]:
# For Hyderabad and HP_D_HO segment

w34 = df_temp[(df_temp['Segment']=='HP_D_HO') & (df_temp['week']==34)]
w35 = df_temp[(df_temp['week']==35)]
w34 = w34.merge(w35, how = 'outer', on = 'RIDER', indicator = True)
constant = w34[(w34['_merge']=='both') & (w34['Segment_y']=='HP_D_HO')]
churned = w34[w34['_merge']=='left_only']
upgrade = w34[(w34['_merge']=='both') & (w34['Segment_y'].isin(['UHP_D_UHO','HP_D_UHO']))]
downgrade = w34[(w34['_merge']=='both') & (~w34['Segment_y'].isin(['HP_D_UHO','UHP_D_UHO','HP_D_HO']))]

In [244]:
df_temp[df_temp['Segment'].isin(['UHP_D_UHO','HP_D_UHO','HP_D_HO','MP_D_UHO','MP_D_HO','MP_D_MO','MP_D_LO','LP_D_UHO','LP_D_MO','LP_D_HO','LP_D_LO'])]['RIDER'].nunique()

5.083551561595502

In [402]:
x = df_temp[(df_temp['week']==39) & (df_temp['Segment']=='MP_Intra_HO')]['daily_incentive']
x = x.to_numpy()
x = x[~np.isnan(x)]
np.percentile(x,60)

IndexError: cannot do a non-empty take from an empty axes.

In [439]:
df_temp = df_captains[df_captains['city'] == 'Chennai']
z = pd.DataFrame()
for s in df_temp[(df_temp['week']==39) & (df_temp['Performance'].notna())]['Segment'].unique():
    x = df_temp[(df_temp['week']==39) & (df_temp['Segment']==s)]['Performance']
    x = x.to_numpy()
    x = x[~np.isnan(x)]
    y = pd.DataFrame({"Segment":[s], "p_point":[50], "Value":[np.percentile(x,50)]})
    z = z.append(y)
    y = pd.DataFrame({"Segment":[s], "p_point":[75], "Value":[np.percentile(x,75)]})
    z = z.append(y)
    y = pd.DataFrame({"Segment":[s], "p_point":[80], "Value":[np.percentile(x,80)]})
    z = z.append(y)
    y = pd.DataFrame({"Segment":[s], "p_point":[85], "Value":[np.percentile(x,85)]})
    z = z.append(y)
    y = pd.DataFrame({"Segment":[s], "p_point":[90], "Value":[np.percentile(x,90)]})
    z = z.append(y)
    y = pd.DataFrame({"Segment":[s], "p_point":[95], "Value":[np.percentile(x,95)]})
    z = z.append(y)
z.head()

,Segment,p_point,Value
0,MP_D_MO,50,5.0
0,MP_D_MO,75,6.0
0,MP_D_MO,80,6.0
0,MP_D_MO,85,6.0
0,MP_D_MO,90,7.0


In [440]:
z.to_csv('x.csv', index=False)

In [319]:
df_captains.head()

,RIDER,MEDIAN_DBR,ADBR,NUM_OBS,captain_id,active_days,Performance,Opportunity,Actual_Captain_cancellations,Rider_cancellation,...,login_hours_y,riderid_x,orders_earnings_y,riderid_y,daily_incentive_y,weekly_incentive_y,rider,link_orders_y,delivery_orders_y,other_orders_y
0,573f29249b0ffc283677a933,3.0,1.500000,2,573f29249b0ffc283677a933,2,1.0,4.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5b40a0a3499d526d0aa1b252,0.0,0.102564,5,5b40a0a3499d526d0aa1b252,5,5.0,5.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5b9a82b06636546559dc406f,0.0,0.333333,4,5b9a82b06636546559dc406f,4,3.0,6.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5bf2a235fe82667f3f01336e,0.0,0.000000,1,5bf2a235fe82667f3f01336e,1,2.0,4.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5c21c968d9bacb2f6f9e87cb,0.0,0.666667,2,5c21c968d9bacb2f6f9e87cb,2,2.0,3.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [441]:
df_captains[df_captains['city']=='Chennai'][['RIDER','Segment','week','mobilenumber']].to_csv('x.csv', index=False)

In [15]:
df_captains['x'] = df_captains.apply(lambda x : 'HH' if x['cu_segment']=='HH' else 'Intra_Inter' if any(y in x['Consistency'] for y in ['intra_week','inter_week']) else x['Segment'][:x['Segment'].find('_')], axis = 1)

In [16]:
df_captains['x'].unique()

array(['Intra_Inter', 'MP', 'LP', 'HH', 'HP', 'UHP'], dtype=object)

In [29]:
df_captains.pivot_table(index='week', columns='x', values = 'RPR',aggfunc='mean')#.to_csv('x.csv', index=False)

x,HH,HP,Intra_Inter,LP,MP,UHP
week,,,,,,
34,3.728693,10.911238,1.088746,2.284791,5.490994,18.751678
35,3.585008,10.944271,1.080512,2.310639,5.464120,18.768750
36,3.837981,10.863203,1.084930,2.329889,5.497521,18.877246
37,3.565196,10.883179,1.080744,2.331637,5.431712,18.711180
38,3.603604,10.797880,1.078023,2.329698,5.417423,18.570922
39,3.538881,10.871846,1.076998,2.312991,5.468373,19.233696
40,3.754207,10.939914,1.072030,2.296849,5.507006,18.978927


In [9]:
df_segment[df_segment['week']==40]['city'].unique()

array(['Delhi', 'Hyderabad', 'Mumbai', 'Bangalore', 'Chennai', 'Kolkata',
       'Jaipur'], dtype=object)

In [22]:
df_captains['RPR'].fillna(0, inplace=True)

In [43]:
df_captains[(df_captains['week']==40)]['link_orders'].sum()/7

102946.85714285714

In [26]:
df_captains[(df_captains['week']==40) & (df_captains['x']=='HH')]['RIDER'].nunique()

9329

In [27]:
35023.0/9329

3.7542073105370353

In [40]:
df_captains.head()

,RIDER,MEDIAN_DBR,ADBR,NUM_OBS,captain_id,active_days,Performance,Opportunity,Actual_Captain_cancellations,Rider_cancellation,...,rf_segment,x,rider_y,login_hours,riderid,orders_earnings,rider,link_orders,delivery_orders,other_orders
0,573f29249b0ffc283677a933,3.0,1.500000,2,573f29249b0ffc283677a933,2,1.0,4.0,1.0,1.0,...,Recent-Underperforming,Intra_Inter,573f29249b0ffc283677a933,2.126944,573f29249b0ffc283677a933,104.97,573f29249b0ffc283677a933,3.0,0.0,0.0
1,5b40a0a3499d526d0aa1b252,0.0,0.102564,5,5b40a0a3499d526d0aa1b252,5,5.0,5.0,0.0,0.0,...,Recent-Midperforming,MP,5b40a0a3499d526d0aa1b252,25.701389,5b40a0a3499d526d0aa1b252,1898.64,5b40a0a3499d526d0aa1b252,20.0,20.0,0.0
2,5b9a82b06636546559dc406f,0.0,0.333333,4,5b9a82b06636546559dc406f,4,3.0,6.0,0.0,0.0,...,Recent-Midperforming,LP,5b9a82b06636546559dc406f,21.691944,5b9a82b06636546559dc406f,1083.01,5b9a82b06636546559dc406f,18.0,0.0,0.0
3,5bf2a235fe82667f3f01336e,0.0,0.000000,1,5bf2a235fe82667f3f01336e,1,2.0,4.0,0.0,0.0,...,Recent-Midperforming,LP,5bf2a235fe82667f3f01336e,0.804444,5bf2a235fe82667f3f01336e,86.37,5bf2a235fe82667f3f01336e,2.0,0.0,0.0
4,5c21c968d9bacb2f6f9e87cb,0.0,0.666667,2,5c21c968d9bacb2f6f9e87cb,2,2.0,3.0,1.0,1.0,...,Recent-Midperforming,LP,5c21c968d9bacb2f6f9e87cb,3.645833,5c21c968d9bacb2f6f9e87cb,441.43,5c21c968d9bacb2f6f9e87cb,4.0,0.0,0.0


In [4]:
pip install ipynb-py-convert

  Created wheel for ipynb-py-convert: filename=ipynb_py_convert-0.4.6-py3-none-any.whl size=4624 sha256=f08cec42111516fd81adad7e8245f149e6fecf5c0f0c21b6ff86f6e47907f6e9
  Stored in directory: /home/jovyan/.cache/pip/wheels/49/ee/0a/ff1f946e7969e39aec10a28d84692859084219f27d2ae35119
Successfully built ipynb-py-convert
Note: you may need to restart the kernel to use updated packages.


In [7]:
import ipynb-py-convert

SyntaxError: invalid syntax (<ipython-input-7-8d73c3d2ff9b>, line 1)

In [9]:
ipynb-py-convert quality_selectors_v2_cleanedup.py quality_selectors_v2_cleanedup.ipynb

SyntaxError: invalid syntax (<ipython-input-9-5aaf2aa13dc9>, line 1)